# Introduction

Recently, I came across the nice little library `xmldataset` that allows you to flatten multi-level XML data so that you can process it easily with Pandas.

```
pip install xmldataset
```

In [37]:
import xmldataset
import pandas as pd
xml = ""

with open ("datasets/jacocoTestReport.xml") as f:
    xml = f.read()


profile="""
report
    package
        class
            name = external_dataset:info,prefix:class_
            method
                name = external_dataset:info,prefix:method_
                counter
                    type = dataset:measures
                    missed = dataset:measures
                    covered = dataset:measures
                    __EXTERNAL_VALUE__ = info:class_name:measures info:method_name:measures
                    
"""


output = xmldataset.parse_using_profile(xml,profile)
coverage_data = pd.DataFrame.from_records(output['measures'])
coverage_data.head()

,class_name,covered,method_name,missed,type
0,org/springframework/samples/petclinic/Petclini...,3,<init>,0,INSTRUCTION
1,org/springframework/samples/petclinic/Petclini...,1,<init>,0,LINE
2,org/springframework/samples/petclinic/Petclini...,1,<init>,0,COMPLEXITY
3,org/springframework/samples/petclinic/Petclini...,1,<init>,0,METHOD
4,org/springframework/samples/petclinic/Petclini...,7,onStartup,0,INSTRUCTION


In [44]:
coverage_data['covered'] = pd.to_numeric(coverage_data['covered'])

In [56]:
coverage_data.pivot_table(
    index=["class_name", "method_name"],
    columns=['type'],
    values=["covered", "missed"])

covered  \
type                                                                                BRANCH   
class_name                                         method_name                               
org/springframework/samples/petclinic/Petclinic... <init>                              NaN   
                                                   createRootApplicationContext        NaN   
                                                   createServletApplicationContext     NaN   
                                                   getServletFilters                   NaN   
                                                   getServletMappings                  NaN   
                                                   onStartup                           NaN   
                                                   registerDandelionServlet            NaN   
                                                   registerServletFilter               NaN   
org/springframework/samples/petclinic/model/Bas... <init>                              NaN   
                                                   getId                               NaN   
                                                   isNew                               2.0   
                                                   setId                               NaN   
org/springframework/samples/petclinic/model/Nam... <init>                              NaN   
                                                   getName                             NaN   
                                                   setName                             NaN   
                                                   toString                            NaN   
org/springframework/samples/petclinic/model/Owner  <init>                              NaN   
                                                   addPet                              NaN   
                                                   getAddress                          NaN   
                                                   getCity                             NaN   
                                                   getPet                              5.0   
                                                   getPets                             NaN   
                                                   getPetsInternal                     1.0   
                                                   getTelephone                        NaN   
                                                   setAddress                          NaN   
                                                   setCity                             NaN   
                                                   setPetsInternal                     NaN   
                                                   setTelephone                        NaN   
                                                   toString                            NaN   
org/springframework/samples/petclinic/model/Person <init>                              NaN   
...                                                                                    ...   
org/springframework/samples/petclinic/web/Owner... initUpdateOwnerForm                 NaN   
                                                   processCreationForm                 0.0   
                                                   processFindForm                     3.0   
                                                   processUpdateOwnerForm              1.0   
                                                   setAllowedFields                    NaN   
                                                   showOwner                           NaN   
org/springframework/samples/petclinic/web/PetCo... <init>                              NaN   
                                                   findOwner                           NaN   
                                                   initCreationForm                    NaN   
                                         

# Limitations
Unfortunately, there is currently an important feature missing. Apparently, you 